In [3]:
torch.cuda.empty_cache()

In [1]:
!pip install FlagEmbedding

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.9 kB/s eta 0:00:00a 0:00:04m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 85.5 kB/s eta 0:00:00a 0:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.6 kB/s eta 0:00:00a 0:00:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 109.4 kB/s eta 0:00:0000:0200:12
  Created wheel for FlagEmbedding: filename=FlagEmbedding-1.3.4-py3-none-any.whl size=232544 sha256=dedc8d4e10860133f15c61ac2b08415e185be0fd32f7d5b9b92ac1ea43bd2721
  Stored in directory: /home/ubuntu/.cache/pip/wheels/59/8c/ba/90918a0fe0371cda2d087cffe7eb4bb4495409b6c67873a410
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18965 sha256=56d08d4c31d31b2a61f67a641ee095cee0ed75d9fb3aaab0511d766fbf1e

In [4]:
from FlagEmbedding import BGEM3FlagModel
from FlagEmbedding import FlagReranker
import torch 
from typing import Dict, Optional, List, Tuple

from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoConfig,
    AutoModelForSequenceClassification,
)
from sentence_transformers import CrossEncoder, SentenceTransformer


class BgeM3:
    def __init__(self, model_name: str = "BAAI/bge-m3"):
        # Преобразуем имя модели для использования в имени сервиса
        self.model_name = model_name
        self.ready = False
        self.model = None
        self.gpu = torch.cuda.is_available()
        self.device = "cuda" if self.gpu else "cpu"
        self.load()

    def load(self) -> None:
        # Загружаем модель через SentenceTransformer с указанием устройства
        self.model = SentenceTransformer(self.model_name, device=self.device)
        self.ready = True

    def predict(
        self, request_data: Dict, request_headers: Optional[Dict] = None
    ) -> Dict:
        texts: List[Tuple[str, str]] = request_data["texts"]
        texts_for_model = [text for _, text in texts]

        embeddings = self.model.encode(
            texts_for_model,
            show_progress_bar=False,
            convert_to_tensor=False,
            max_length=1024,
        )

        # Возвращаем эмбеддинги в формате списка
        return {"embeddings": embeddings}

BgeM3()

: 

In [8]:
response = model.predict(
    request_data={
        "texts": [
            ("query", "how much protein should a female eat"),
            ("query", "南瓜的家常做法"),
            (
                "passage",
                "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
            ),
            (
                "passage",
                "1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅",
            ),
        ]
    }
)

In [9]:
len(response["embeddings"])

4

In [5]:
request_data = {
    "texts": [
        ("query", "how much protein should a female eat"),
        ("query", "南瓜的家常做法"),
        (
            "passage",
            "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
        ),
        (
            "passage",
            "1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅",
        ),
    ]
}

In [10]:
import requests

response = requests.post(
    "http://localhost:8100/v1/models/BAAI_bge_reranker_v2_m3:predict",
    json=request_data,
)

In [11]:
response

<Response [200]>

In [12]:
response.content

b'{"scores":[-7.6122660636901855,-8.917508125305176,-10.979835510253906,-7.614435195922852]}'

In [20]:
for embed in response.json()["embeddings"]:
    print(embed[:10])

[0.02018592692911625, 0.011204439215362072, -0.04514234513044357, -0.0394994355738163, 0.017567913979291916, -0.025198882445693016, -0.025208625942468643, 0.06422241777181625, 0.03213871642947197, -0.026800913736224174]
[0.04857625067234039, 0.0593828447163105, 0.003986190538853407, -0.020708534866571426, 0.024149436503648758, -0.014857766218483448, -0.0005905894795432687, 0.05966014415025711, 0.019906768575310707, -0.013999292626976967]
[0.03266947343945503, 0.004132492933422327, -0.05025511234998703, -0.021615885198116302, 0.03252109885215759, -0.04739893600344658, -0.03377801179885864, 0.04409226030111313, 0.04651404544711113, -0.051693595945835114]
[0.04768911749124527, 0.05657833069562912, 0.009223462082445621, -0.018913310021162033, 0.01720563881099224, -0.011063066311180592, 0.014807222411036491, 0.05328642204403877, 0.0010438720928505063, -0.012906507588922977]


In [1]:
import sentence_transformers

model = sentence_transformers.SentenceTransformer(
    "intfloat/multilingual-e5-large-instruct",
    prompts={
        "query": "Instruct: Given a web search query, retrieve relevant passages that answer the query\nQuery: "
    },
    device="cpu",
)

/Users/mvdremin/miniforge3/envs/gigachain/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
embeddings = model.encode(
    [text[1] for text in request_data["texts"]],
    show_progress_bar=True,
    normalize_embeddings=True,
    prompt_name="query",
)
for embed in embeddings.tolist():
    print(embed[:10])

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.05s/it]

[0.020185990259051323, 0.011204462498426437, -0.04514240100979805, -0.03949938341975212, 0.01756785623729229, -0.02519889548420906, -0.025208545848727226, 0.06422246247529984, 0.03213874623179436, -0.026800911873579025]
[0.048576340079307556, 0.05938291922211647, 0.003986139316111803, -0.020708562806248665, 0.02414940483868122, -0.014857753179967403, -0.0005905400612391531, 0.05966009199619293, 0.019906774163246155, -0.013999267481267452]
[0.02288028784096241, 0.00972859188914299, -0.04084709659218788, -0.027168048545718193, 0.03248666226863861, -0.044196922332048416, -0.028844982385635376, 0.0638773962855339, 0.04096970334649086, -0.04006381332874298]
[0.04198748245835304, 0.0473160557448864, 0.004951199982315302, -0.026837505400180817, 0.029732720926404, -0.012284206226468086, 0.01857127994298935, 0.055720094591379166, 0.012999649159610271, -0.0016542201628908515]


In [11]:
embeddings = model.embed_documents(
    [text[1] for text in request_data["texts"]],
)
for embed in embeddings:
    print(embed[:10])

[0.020185990259051323, 0.011204462498426437, -0.04514240100979805, -0.03949938341975212, 0.01756785623729229, -0.02519889548420906, -0.025208545848727226, 0.06422246247529984, 0.03213874623179436, -0.026800911873579025]
[0.048576340079307556, 0.05938291922211647, 0.003986139316111803, -0.020708562806248665, 0.02414940483868122, -0.014857753179967403, -0.0005905400612391531, 0.05966009199619293, 0.019906774163246155, -0.013999267481267452]
[0.02288028784096241, 0.00972859188914299, -0.04084709659218788, -0.027168048545718193, 0.03248666226863861, -0.044196922332048416, -0.028844982385635376, 0.0638773962855339, 0.04096970334649086, -0.04006381332874298]
[0.04198748245835304, 0.0473160557448864, 0.004951199982315302, -0.026837505400180817, 0.029732720926404, -0.012284206226468086, 0.01857127994298935, 0.055720094591379166, 0.012999649159610271, -0.0016542201628908515]


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

model = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct",
    model_kwargs={
        "prompts": {
            "query": "Instruct: Given a web search query, retrieve relevant passages that answer the query\nQuery: "
        },
        "device": "cpu",
    },
    encode_kwargs={"normalize_embeddings": True, "prompt_name": "query"},
)

embeddings = model.embed_documents(
    [text[1] for text in request_data["texts"]],
)
for embed in embeddings:
    print(embed[:10])

In [37]:
embeddings = model.encode(
    [text[1] for text in request_data["texts"]], show_progress_bar=True
)
for embed in embeddings.tolist():
    print(embed[:10])

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.09it/s]

[0.02380332164466381, 0.0233569648116827, -0.04431702941656113, -0.03123355843126774, 0.010486085899174213, -0.02713814191520214, -0.020534325391054153, 0.04382570832967758, 0.020839078351855278, -0.02268272452056408]
[0.05234979838132858, 0.06129227578639984, 0.004823273979127407, -0.02078615128993988, 0.02621222846210003, -0.011689964681863785, 0.0046771918423473835, 0.05655382201075554, 0.020846951752901077, -0.02351178228855133]
[0.03266941010951996, 0.004132444504648447, -0.050254981964826584, -0.021615881472826004, 0.03252111002802849, -0.047398969531059265, -0.03377801552414894, 0.04409223794937134, 0.04651397466659546, -0.051693618297576904]
[0.04768915846943855, 0.05657828971743584, 0.009223384782671928, -0.018913382664322853, 0.017205655574798584, -0.011062991805374622, 0.014807027764618397, 0.053286388516426086, 0.0010437833843752742, -0.012906370684504509]


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings


embeddings = HuggingFaceEmbeddings(model_name=EMBEDDER_MODEL_NAME)

Milvus(
    embedding_function=embeddings,
    connection_args={"uri": path},
)

NameError: name 'EMBEDDER_MODEL_NAME' is not defined

In [12]:
from langchain_community.retrievers import ElasticSearchBM25Retriever

In [ ]:
ElasticSearchBM25Retriever()